In [ ]:
!pip install yake numpy 

In [ ]:
# import block
import yake
import pandas as pd
import os
import glob
import csv
import itertools
import string
import pickle

In [ ]:
kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram = 1
deduplication_threshold = 0.9
nKeywords = 5
customExtractor = yake.KeywordExtractor(lan=language, n = max_ngram, dedupLim = deduplication_threshold, top = nKeywords, features=None)

# descriptors = pd.read_csv("/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv")
# descriptors[0:10]

fnames = glob.glob("/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/*.jpg")

print(f"{len(fnames)} images")

comments = pd.read_csv("/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv", sep="|")


In [ ]:
comments

In [ ]:
descriptions = {}

for i in range(len(comments)):
    if comments.loc[i, "image_name"] in descriptions:
        descriptions[comments.loc[i, "image_name"]].append(comments.loc[i, " comment"])
    else:
        descriptions[comments.loc[i, "image_name"]] = [comments.loc[i, " comment"]]
        
        
print("consolidating complete")

print("first 10 ")
for x in itertools.islice(descriptions.items(), 10):
    print(x)
    print("\n")
    
sampleSet = []
testing = True

if testing:
    for key in itertools.islice(descriptions.keys(), 250):
        sampleSet.append((key, descriptions[key]))

    print("sampleset of 100 imgs 5 captions each created")
else:
    for key in descriptions.keys():
        sampleSet.append((key, descriptions[key]))
        
    print("all items added to sampleSet")

In [ ]:
# YAKE params
language = 'en'
max_ngram = 1
deduplication_thresh = 0.9
deduplication_algo = 'seqm'
winSize = 1
# nKeywords = 10 -> dependent on caption length, try for approx 1/3 caption length and see?

In [ ]:
# TODO: this works for keyword extraction for now, but redo later to get more relevant/searchable keywords?
# i.e. first image 1000092795.jpg, people won't search for just 'green' or 'enjoy' but they would search for 'man' 'garden' 'bushes'
# create a dictionary with these types of highly searchable words (WHAT DO THEY HAVE IN COMMON) and make keywords more USEFUL based on that

sampleKw = []
for item in sampleSet:
    key, caps = item
    keywords = []
    for cap in caps:
        cap = " ".join(cap.lower().translate(str.maketrans('', '', string.punctuation)).split())
        nKeywords = int(len(cap.split()) // 3)

        kw_extr = yake.KeywordExtractor(
            lan = language,
            n = max_ngram,
            dedupLim = deduplication_thresh,
            dedupFunc = deduplication_algo,
            windowsSize = winSize,
            top = nKeywords, 
            features = None
        )
        kw = kw_extr.extract_keywords(cap)
        # extracted in format (word, relevance_score) where lower relevance_score -> higher relevance to string

        for w in kw: keywords.append(w)
    
    keywords.sort(key=lambda x: x[1])
    isoKw = set([kw[0] for kw in keywords])
    finWords = [x for x in list(isoKw)[0:7]]
    sampleKw.append((key, finWords))
#     print(f"image: {key} | found keywords: {finWords}")
    
print(len(sampleKw))
    
filename="250kw.pkl"
pickle.dump(sampleKw, open(filename, 'wb'))

print("data dumped to /kaggle/working/flickrKeywords.pkl")
